In [ ]:
%load_ext autoreload
%autoreload 2
import transformers
import torch
import math
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from mcllm.model.llm import *
from tqdm import tqdm

In [ ]:
def create_low_rank_matrix(m, n, rank):
    A = np.random.randn(m, rank) @ np.random.randn(rank, n)
    return A


mat = create_low_rank_matrix(10, 10, rank=3)

In [16]:
# single prediction

seq_len = 100
llm = NanoBERT(vocab_size=100, max_seq_len=seq_len)
batch_size = 1
x = torch.LongTensor(np.zeros((batch_size, seq_len)))
out = llm(x)
print('shapes', x.shape, out.shape)

shapes torch.Size([1, 100]) torch.Size([1, 100])


In [15]:
# train on 1 example
device = 'cuda'
num_epochs = 10
lr = 1e-2
llm = llm.to(device)
optimizer = torch.optim.Adam(llm.parameters(), lr=lr)
llm.train()
for i in range(num_epochs):
    print(f'Epoch: {i + 1}')
    train_loss = 0.0
    train_preds = []
    train_labels = []

    llm.train()
    # for step, batch in enumerate(tqdm(dataloader.get_split('train'), total=dataloader.steps('train'))):
    x = torch.LongTensor(np.zeros((batch_size, seq_len))).to(device)
    pred = llm(x)
    loss = F.mse_loss(x.float(), pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss.item()

    val_loss = 0.0
    val_preds = []
    val_labels = []

    # bert.eval()
    # for step, batch in enumerate(tqdm(dataloader.get_split('val'), total=dataloader.steps('val'))):
    #     logits = bert(batch['input_ids'].to(device))

    #     probs = F.softmax(logits[:, 0, :], dim=-1).cpu()
    #     pred = torch.argmax(probs, dim=-1)  # (B)
    #     val_preds += pred.detach().tolist()
    #     val_labels += [l.item() for l in batch['label_ids']]

    #     loss = F.cross_entropy(logits[:, 0, :].cpu(), batch['label_ids'])

    #     val_loss += loss.item()

    # print()
    print(f'Train loss {train_loss}')
    #     f'Train loss: {train_loss / dataloader.steps("train")} | Val loss: {val_loss / dataloader.steps("val")}')
    # print(
    #     f'Train acc: {accuracy_score(train_labels, train_preds)} | Val acc: {accuracy_score(val_labels, val_preds)}')
    # print(
    #     f'Train f1: {f1_score(train_labels, train_preds)} | Val f1: {f1_score(val_labels, val_preds)}')

Epoch: 1
Train loss 0.002657452132552862
Epoch: 2
Train loss 0.007548022549599409
Epoch: 3
Train loss 0.0009820930426940322
Epoch: 4
Train loss 0.00149148132186383
Epoch: 5
Train loss 0.004250728990882635
Epoch: 6
Train loss 0.0023586226161569357
Epoch: 7
Train loss 0.0003296678769402206
Epoch: 8
Train loss 0.0003778900427278131
Epoch: 9
Train loss 0.0017221131129190326
Epoch: 10
Train loss 0.0020630392245948315
